In [ ]:
from glob import glob 
import pandas as pd
import csv
import numpy as np
import chardet 
import os

In [ ]:
SST_filepath = '../../SST_Data/recc'

In [ ]:
subs = list(set([i.split('-')[1].split('/')[0] for i in glob(SST_file_path + '/sub-*/ses-baselineYear1Arm1/func/*SST*')]))

In [ ]:
no_event = []
event_files = []
for sub in subs: 
        file = glob(f'../../SST_data/recc/sub-{sub}/ses-baselineYear1Arm1/func/*SST*')
       # file = glob(f'../../SST_data/recc/sub-NDAR{sub}/ses-baselineYear1Arm1/func/*SST*')
        if len(file) > 0: 
            event_files.append(str(file[0]))
        else: 
            no_event.append(sub)

In [ ]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
pd.option_context('display.max_rows', 999)
pd.options.display.max_rows = 999

In [ ]:
def get_encoding(file): 
    """
    retrieves character encoding of file to be passed to 
    pd.read_csv()
    """
    with open(file, mode = "rb") as f: 
        rawdata = f.read()
        return chardet.detect(rawdata)['encoding']
    
def Diff(li1, li2):
    """
    returns differenes in two lists, from geeks 4 geeks 
    """
    return (list(set(li1) - set(li2))) 

the files sometimes have odd encoding, or an extra column or two or three, requiring some nested try/except syntax

In [ ]:
from pandas.io.parsers import ParserError

dfp = []
for file in event_files: 
        encoding = get_encoding(file)
        if file[-3:] == 'txt': #this could be put in a function to be neater
            try: 
                df = pd.read_csv(file, sep = '\t', encoding = encoding)
                if df.columns[0] != 'ExperimentName': #to fix header differences
                     df = pd.read_csv(file, sep = '\t', skiprows=[0], encoding = encoding)
            except: 
                try: 
                    df = pd.read_csv(file, sep = '\t', skiprows=[0,1], encoding = encoding)
                except: 
            #these files specifically are encoded oddly
            if file == '../../SST_data/recc/sub-NDARINVB1CK69PR/ses-baselineYear1Arm1/func/ABCD-SST-fMRI_run-20161105142951-EventRelatedInformation.txt' \
            or file == '../../SST_data/recc/sub-NDARINVB1CK69PR/ses-baselineYear1Arm1/func/ABCD-SST-fMRI_run-20161105142323-EventRelatedInformation.txt':  
                df = pd.read_csv(file, sep = '\t', skiprows=[0,1,2], encoding = encoding)

        elif file[-3:] == 'csv': 
            df = pd.read_csv(file, encoding = encoding)
            if df.columns[0] != 'ExperimentName': 
                df = pd.read_csv(file, encoding = encoding, skiprows=[0])
                if len(df.columns) == 1:
                    try: 
                        df = pd.read_csv(file, encoding = encoding, skiprows=[0], sep='\t')
                    except ParserError: 
                        print(event_file)

                    
        dfp.append(df)
print("concatting files together") 
event_df = pd.concat(dfp, sort=True, axis = 0)


get rid of all of the non-SST files

In [ ]:
print(len(event_df['NARGUID'].loc[event_df['ExperimentName'] == 'MID_20171130'].unique()))
event_df = event_df.loc[~(event_df['ExperimentName'] == 'MID_20171130')] 

print(len(event_df['NARGUID'].loc[event_df['ExperimentName'] == 'MID_20161218'].unique()))
event_df = event_df.loc[~(event_df['ExperimentName'] == 'MID_20161218')]

print(len(event_df['NARGUID'].loc[(event_df['ExperimentName'] == 'ABCD_SST_Practice_CDbox_20160729')].unique())) #these will need to be counted
event_df = event_df.loc[~(event_df['ExperimentName'] == 'ABCD_SST_Practice_CDbox_20160729')] #these will need to be counted

event_df = event_df.dropna(axis = 1, how='all')

get rid of cluttery columns 

In [ ]:
for col in event_df.columns:
    if 'Unnamed' in col or 'task' in col or 'Test' in col or 'Wait' in col or 'Session' in col or 'Run' in col or 'VARIABLE' in col:
        del event_df[col] 

In [ ]:
event_df.to_csv('raw_concat_jan14.csv')

Now, go to clean_SST and run raw_concat through that for further processing. 